### Import of required libraries

In [94]:
import pandas as pd
from traffic.core import Traffic

In [95]:
pd.set_option("display.max_columns", None)

### Import Dataset and Trajectory Snippet Dataset

In [96]:
table = pd.read_csv('Final_Table.csv')
trajs = pd.read_parquet('Trajectory_Snippet.parquet')

### 1. Add Trajectory Data at the Predction Point to the table

Since we only use trajectory snippets for the AROT prediction model, we add the trajectory-related data of the predicting flight to the table in order to be able to use it in the RET prediction model if needed.

In [97]:
# If needed: convert time columns in both DataFrames to datetime
table['Time of Prediction'] = pd.to_datetime(table['Time of Prediction'])
trajs['timestamp'] = pd.to_datetime(trajs['timestamp'])

# Merge by Flight ID and time
merged = pd.merge(
    table,
    trajs[['flight_id', 'timestamp', 'groundspeed', 'geoaltitude', 'vertical_rate', 'distance']],
    left_on=['Flight ID', 'Time of Prediction'],
    right_on=['flight_id', 'timestamp'],
    how='left'  # keep all rows from 'table' even if there is no match in 'trajs'
)

# Rename columns for clarity
merged = merged.rename(columns={
    'groundspeed': 'groundspeed at prediction',
    'geoaltitude': 'geoaltitude at prediction',
    'vertical_rate': 'vertical_rate at prediction',
    'distance': 'distance at prediction'
})

# Optional: drop helper columns that are no longer needed
merged = merged.drop(columns=['flight_id', 'timestamp'])

# Final result: your extended 'table'
table = merged


### 2. Add MeteoSwiss Weather Data to the Table

The original Table used Weather Data from METARS (opensky) and openmeteoapi. Since May 2025, MeteoSwiss has been publishing open historical station data (e.g., SwissMetNet) with more variables and a higher sampling rate (often 10-minute intervals).

[MeteoSwiss Open Data](https://www.meteoswiss.admin.ch/services-and-publications/service/open-data.html)



The weather data were downloaded from [MeteoSwiss Open Data File Downloader](https://www.meteoswiss.admin.ch/services-and-publications/applications/ext/download-data-without-coding-skills.html#lang=en&mdt=normal&pgid=&sid=&col=&di=&tr=&hdr=)
 for the automatic weather station at Zurich Airport (Kloten) with a 10-minute time resolution. Two files were downloaded: the first covers 2020-01-01 to 2024-12-31, and the second covers 2025-01-01 up to the day before the download.

File 1: https://data.geo.admin.ch/ch.meteoschweiz.ogd-smn/klo/ogd-smn_klo_t_historical_2020-2029.csv

File 2: https://data.geo.admin.ch/ch.meteoschweiz.ogd-smn/klo/ogd-smn_klo_t_recent.csv

In [98]:
# Read first CSV; treat empty strings or ';;' as missing values
df = pd.read_csv("ogd-smn_klo_t_historical_2020-2029.csv", sep=";", na_values=["", ";;"])

# Optionally: parse the date column as datetime (day-first format)
df["reference_timestamp"] = pd.to_datetime(df["reference_timestamp"], dayfirst=True)

# Read second CSV with the same settings
df2 = pd.read_csv("ogd-smn_klo_t_recent.csv", sep=";", na_values=["", ";;"])

# Optionally: parse the date column as datetime (day-first format)
df2["reference_timestamp"] = pd.to_datetime(df2["reference_timestamp"], dayfirst=True)

meteo = pd.concat([df, df2], ignore_index=True)

#### Rename columns to more descriptive names using the metadata CSV

In [99]:
column_rename_map = {
    'station_abbr': 'station',
    'tre200s0': 'temp_2m',
    'tre005s0': 'temp_5cm',
    'tresurs0': 'temp_surface',
    'xchills0': 'temp_chill',
    'ure200s0': 'rh_2m',
    'tde200s0': 'dewpoint_2m',
    'pva200s0': 'vapour_pressure',
    'prestas0': 'pressure_qfe',
    'pp0qnhs0': 'pressure_qnh',
    'pp0qffs0': 'pressure_qff',
    'ppz850s0': 'geopot_850',
    'ppz700s0': 'geopot_700',
    'fkl010z1': 'gust_1s_ms',
    'fve010z0': 'wind_vec_ms',
    'fkl010z0': 'wind_scalar_ms',
    'dkl010z0': 'wind_dir',
    'wcc006s0': 'foehn_idx',
    'fu3010z0': 'wind_scalar_kmh',
    'fkl010z3': 'gust_3s_ms',
    'fu3010z1': 'gust_1s_kmh',
    'fu3010z3': 'gust_3s_kmh',
    'rre150z0': 'precip_10min',
    'htoauts0': 'snow_depth',
    'gre000z0': 'rad_global',
    'ods000z0': 'rad_diffuse',
    'oli000z0': 'rad_lw_in',
    'olo000z0': 'rad_lw_out',
    'osr000z0': 'rad_sw_reflect',
    'sre000z0': 'sunshine_10min'
}


In [100]:
meteo.rename(columns=column_rename_map, inplace=True)

In [101]:
meteo

,station,reference_timestamp,temp_2m,temp_5cm,temp_surface,temp_chill,rh_2m,dewpoint_2m,vapour_pressure,pressure_qfe,pressure_qnh,pressure_qff,geopot_850,geopot_700,gust_1s_ms,wind_vec_ms,wind_scalar_ms,wind_dir,foehn_idx,wind_scalar_kmh,gust_3s_ms,gust_1s_kmh,gust_3s_kmh,precip_10min,snow_depth,rad_global,rad_diffuse,rad_lw_in,rad_lw_out,rad_sw_reflect,sunshine_10min
0,KLO,2020-01-01 00:00:00,-1.7,-1.3,-0.9,-3.9,96.8,-2.1,5.2,982.5,1033.7,1036.4,NaN,NaN,2.5,1.5,1.6,11.0,NaN,5.8,2.4,9.0,8.6,0.0,NaN,2,1.0,305,NaN,NaN,0
1,KLO,2020-01-01 00:10:00,-1.6,-1.3,-0.9,-3.6,96.5,-2.1,5.2,982.3,1033.5,1036.2,NaN,NaN,2.0,1.5,1.5,6.0,NaN,5.4,1.9,7.2,6.8,0.0,NaN,2,1.0,305,NaN,NaN,0
2,KLO,2020-01-01 00:20:00,-1.4,-1.2,-0.9,-4.1,96.4,-1.9,5.3,982.3,1033.5,1036.1,NaN,NaN,2.9,2.0,2.0,18.0,NaN,7.2,2.8,10.4,10.1,0.0,NaN,2,1.0,306,NaN,NaN,0
3,KLO,2020-01-01 00:30:00,-1.6,-1.2,-0.8,-4.2,96.8,-2.0,5.3,982.3,1033.5,1036.2,NaN,NaN,2.7,1.9,1.9,23.0,NaN,6.8,2.7,9.7,9.7,0.0,NaN,2,1.0,307,NaN,NaN,0
4,KLO,2020-01-01 00:40:00,-1.6,-1.3,-0.9,-4.3,96.5,-2.1,5.2,982.2,1033.3,1036.1,NaN,NaN,3.1,1.9,2.0,47.0,NaN,7.2,3.0,11.2,10.8,0.0,NaN,2,1.0,299,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302683,KLO,2025-10-02 23:10:00,3.3,-0.1,-0.5,1.7,96.5,2.8,7.5,973.9,1024.7,1026.3,NaN,NaN,2.4,1.6,1.7,345.0,NaN,6.1,2.3,8.6,8.3,0.0,0.0,3,2.0,253,NaN,NaN,0
302684,KLO,2025-10-02 23:20:00,4.1,-0.6,-1.1,4.1,94.8,3.3,7.8,973.8,1024.6,1026.1,NaN,NaN,1.9,0.9,1.0,351.0,NaN,3.6,1.9,6.8,6.8,0.0,0.0,2,1.0,253,NaN,NaN,0
302685,KLO,2025-10-02 23:30:00,3.3,-1.0,-1.5,3.3,93.9,2.4,7.3,973.7,1024.5,1026.1,NaN,NaN,2.0,1.2,1.2,324.0,NaN,4.3,1.9,7.2,6.8,0.0,0.0,2,1.0,253,NaN,NaN,0
302686,KLO,2025-10-02 23:40:00,4.9,-0.9,-1.5,4.9,89.0,3.2,7.7,973.7,1024.5,1025.8,NaN,NaN,1.9,0.9,1.0,316.0,NaN,3.6,1.8,6.8,6.5,0.0,0.0,2,1.0,253,NaN,NaN,0


Add the new weather data to the table. Each flight receives the weather data from the most recent measurement prior to the time of prediction.

In [102]:
# Ensure both time columns are interpreted as timezone-aware datetimes (UTC)
table["Time of Prediction"] = pd.to_datetime(table["Time of Prediction"], utc=True)
meteo["reference_timestamp"] = pd.to_datetime(meteo["reference_timestamp"], utc=True)

# Sort both DataFrames (required for merge_asof)
table = table.sort_values("Time of Prediction").reset_index(drop=True)
meteo = meteo.sort_values("reference_timestamp").reset_index(drop=True)


# Merge: for each prediction timestamp, take the most recent measurement at or before that time
table = pd.merge_asof(
    table,
    meteo,
    left_on="Time of Prediction",
    right_on="reference_timestamp",
    direction="backward"
)


Compute Head- and Crosswind components with the new data

In [103]:
from traffic.data import airports
import numpy as np

In [104]:
# Runway bearing in degrees
runway_direction = airports['LSZH'].runways.data.bearing.loc[2]
runway_rad = np.radians(runway_direction)

# Wind speed in knots; wind direction in degrees
table['new_wind_speed_kt'] = table['wind_scalar_kmh'] / 1.852

# Relative angle (wind direction minus runway heading), in radians
angle = np.radians(table['wind_dir']) - runway_rad

# Headwind component in knots (positive = headwind, negative = tailwind)
table['new_headwind_kt'] = (table['new_wind_speed_kt'] * np.cos(angle)).round(2)

# Crosswind component in knots (signed)
table['new_crosswind_kt'] = (table['new_wind_speed_kt'] * np.sin(angle)).round(2)


In [105]:
table

,Flight ID,Entry Time,Exit Time,ICAO Code,A/C Type,ICAO Aircraft Type,Propulsion Type,Number of Engines,MALW [kg],ICAO Weight Turbulence Category,Gate Region,RET,ROT [s],Time of Prediction,Wind speed [kt],Wind direction [°],Visibility Category,Temperature [°C],Meteo idx,Precipitation [mm],Precipitation idx,Precipitation Timestamp,Hour sin,Hour cos,Minute sin,Minute cos,Day of week sin,Day of week cos,Month sin,Month cos,Year sin,Year cos,Day/Night,No Wind,Wind Variable,Wind direction sin,Wind direction cos,Headwind [kt],Crosswind [kt],Traffic Intensity RWY 10,Traffic Intensity RWY 14,Departure Traffic Intensity RWY 16,Arrival Traffic Intensity RWY 16,Traffic Intensity RWY 16,Departure Traffic Intensity RWY 28,Arrival Traffic Intensity RWY 28,Traffic Intensity RWY 28,Traffic Intensity RWY 32,Departure Traffic Intensity RWY 34,Arrival Traffic Intensity RWY 34,Traffic Intensity RWY 34,Airport Departure Traffic Intensity,Airport Arrival Traffic Intensity,Total Airport Traffic Intensity,Time Reserve [s],Number of preceding Traffic,Preceding Traffic ICAO Code,Preceding Traffic A/C Type,Preceding Traffic ICAO Aircraft Type,Preceding Traffic Propulsion Type,Preceding Traffic Number of Engines,Preceding Traffic MALW [kg],Preceding Traffic ICAO Weight Turbulence Category,Preceding Traffic Gate,Preceding Aircraft Geoaltitude,Preceding Aircraft Speed,Distance to Preceding Aircraft,Number of Successive Traffic,Successive Traffic ID,Successive Traffic A/C Type,Successive Traffic Geoaltitude,Successive Traffic Groundspeed,Successive Traffic Track,Successive Traffic Vertical Rate,Successive Traffic Distance,Successive Traffic Track sin,Successive Traffic Track cos,Successor Aligned,ROT Previous Flight [s],Average ROT Previous 5 Flight [s],Average ROT Previous Flight (Intensity) [s],ROT Previous Flight same A/C Type [s],Average ROT Previous 5 Flights same A/C Type [s],Average ROT Previous Flight same A/C Type (Intensity) [s],Average ROT Previous Flight same A/C Type Total [s],ROT Previous Flight same ICAO Weight Category [s],Average ROT Previous 5 Flights same ICAO Weight Category [s],Average ROT Previous Flight same ICAO Weight Category (Intensity) [s],Average ROT Previous Flight same ICAO Weight Category Total [s],Average ROT Total [s],groundspeed at prediction,geoaltitude at prediction,vertical_rate at prediction,distance at prediction,station,reference_timestamp,temp_2m,temp_5cm,temp_surface,temp_chill,rh_2m,dewpoint_2m,vapour_pressure,pressure_qfe,pressure_qnh,pressure_qff,geopot_850,geopot_700,gust_1s_ms,wind_vec_ms,wind_scalar_ms,wind_dir,foehn_idx,wind_scalar_kmh,gust_3s_ms,gust_1s_kmh,gust_3s_kmh,precip_10min,snow_depth,rad_global,rad_diffuse,rad_lw_in,rad_lw_out,rad_sw_reflect,sunshine_10min,new_wind_speed_kt,new_headwind_kt,new_crosswind_kt
0,SIA346_291674,2024-03-01 06:23:00+00:00,2024-03-01 06:23:57+00:00,SIA,B77W,L2J,Jet,2.0,251290.0,Heavy,North,H1,57.0,2024-03-01 06:21:26+00:00,1.0,0.0,3,5.0,16.0,0.6,30.0,2024-03-01 06:00:00+00:00,1.0,0.5,0.904508,0.206107,0.283058,0.049516,1.0,0.5,0.999013,0.531395,Day,False,True,0.000000,0.00000,1.00,1.00,0,0,0,0,0,7,0,7,0,0,0,0,7,0,7,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1325.000000,0.0,13.752771,0.0,NaN,NaN,16000.0,417.0,317.231857,0.0,19.097551,0.000000,0.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.0,2950.0,-896.0,4.003528,KLO,2024-03-01 06:20:00+00:00,5.1,4.4,4.6,5.1,97.1,4.7,8.5,959.0,1009.2,1010.3,NaN,NaN,1.5,1.1,1.1,106.0,NaN,4.0,1.5,5.4,5.4,0.4,0.0,2,1.0,336,NaN,NaN,0,2.159827,1.85,-1.12
1,ETH736_1162,2024-03-01 06:32:42+00:00,2024-03-01 06:33:36+00:00,ETH,A359,L2J,Jet,2.0,207000.0,Heavy,South,H1,54.0,2024-03-01 06:31:12+00:00,1.0,0.0,3,5.0,16.0,0.6,30.0,2024-03-01 06:00:00+00:00,1.0,0.5,0.447736,0.002739,0.283058,0.049516,1.0,0.5,0.999013,0.531395,Day,False,True,0.000000,0.00000,1.00,1.00,0,1,0,0,0,9,0,9,0,0,0,0,9,1,10,525.0,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1325.000000,0.0,13.752771,0.0,NaN,NaN,16000.0,417.0,317.231857,0.0,19.097551,0.000000,0.000000,False,57.0,5

### 3. Add a RET Trend Feature

In [106]:
import pandas as pd

# Ensure datetime parsing for both timestamp columns (UTC recommended for consistency)
table["Time of Prediction"] = pd.to_datetime(table["Time of Prediction"], utc=True, errors="coerce")
table["Exit Time"] = pd.to_datetime(table["Exit Time"], utc=True, errors="coerce")

# One-hot style indicators for RET categories
table["is_H1"] = (table["RET"] == "H1").astype(int)
table["is_H2"] = (table["RET"] == "H2").astype(int)
table["is_H3"] = (table["RET"] == "H3").astype(int)

# --- Build cumulative counts over time (by 'Exit Time') ---
events = (
    table.sort_values("Exit Time")
         .assign(
             H1_cum=lambda x: x["is_H1"].cumsum(),
             H2_cum=lambda x: x["is_H2"].cumsum(),
             H3_cum=lambda x: x["is_H3"].cumsum()
         )[["Exit Time", "H1_cum", "H2_cum", "H3_cum"]]
)

# --- Cumulative counts up to the prediction time ---
# merge_asof requires both frames to be sorted by the merge keys
table = table.sort_values("Time of Prediction").reset_index(drop=True)

table = pd.merge_asof(
    table,
    events,
    left_on="Time of Prediction",
    right_on="Exit Time",
    direction="backward",          # take the most recent event at or before prediction time
    suffixes=("", "_evt")          # avoid _x/_y; we'll drop the right-side timestamp after merge
).rename(columns={"H1_cum": "H1_upto", "H2_cum": "H2_upto", "H3_cum": "H3_upto"})

# Drop the duplicated 'Exit Time' coming from events (keep the original one)
if "Exit Time_evt" in table.columns:
    table = table.drop(columns=["Exit Time_evt"])

# --- Cumulative counts up to (prediction time - 30 minutes) ---
table["pred_minus_30min"] = table["Time of Prediction"] - pd.Timedelta("30min")

table = pd.merge_asof(
    table,
    events,
    left_on="pred_minus_30min",
    right_on="Exit Time",
    direction="backward",
    suffixes=("", "_evt")
).rename(columns={"H1_cum": "H1_before", "H2_cum": "H2_before", "H3_cum": "H3_before"})

# Drop the duplicated 'Exit Time' again and the helper column
if "Exit Time_evt" in table.columns:
    table = table.drop(columns=["Exit Time_evt"])
table = table.drop(columns=["pred_minus_30min"])

# --- Window counts = difference between the two cumulative snapshots (30-minute window) ---
table["H1"] = (table["H1_upto"].fillna(0) - table["H1_before"].fillna(0)).astype(int)
table["H2"] = (table["H2_upto"].fillna(0) - table["H2_before"].fillna(0)).astype(int)
table["H3"] = (table["H3_upto"].fillna(0) - table["H3_before"].fillna(0)).astype(int)

# Clean up helper columns
table = table.drop(columns=[
    "is_H1", "is_H2", "is_H3",
    "H1_upto", "H2_upto", "H3_upto",
    "H1_before", "H2_before", "H3_before"
])


In [107]:
table

,Flight ID,Entry Time,Exit Time,ICAO Code,A/C Type,ICAO Aircraft Type,Propulsion Type,Number of Engines,MALW [kg],ICAO Weight Turbulence Category,Gate Region,RET,ROT [s],Time of Prediction,Wind speed [kt],Wind direction [°],Visibility Category,Temperature [°C],Meteo idx,Precipitation [mm],Precipitation idx,Precipitation Timestamp,Hour sin,Hour cos,Minute sin,Minute cos,Day of week sin,Day of week cos,Month sin,Month cos,Year sin,Year cos,Day/Night,No Wind,Wind Variable,Wind direction sin,Wind direction cos,Headwind [kt],Crosswind [kt],Traffic Intensity RWY 10,Traffic Intensity RWY 14,Departure Traffic Intensity RWY 16,Arrival Traffic Intensity RWY 16,Traffic Intensity RWY 16,Departure Traffic Intensity RWY 28,Arrival Traffic Intensity RWY 28,Traffic Intensity RWY 28,Traffic Intensity RWY 32,Departure Traffic Intensity RWY 34,Arrival Traffic Intensity RWY 34,Traffic Intensity RWY 34,Airport Departure Traffic Intensity,Airport Arrival Traffic Intensity,Total Airport Traffic Intensity,Time Reserve [s],Number of preceding Traffic,Preceding Traffic ICAO Code,Preceding Traffic A/C Type,Preceding Traffic ICAO Aircraft Type,Preceding Traffic Propulsion Type,Preceding Traffic Number of Engines,Preceding Traffic MALW [kg],Preceding Traffic ICAO Weight Turbulence Category,Preceding Traffic Gate,Preceding Aircraft Geoaltitude,Preceding Aircraft Speed,Distance to Preceding Aircraft,Number of Successive Traffic,Successive Traffic ID,Successive Traffic A/C Type,Successive Traffic Geoaltitude,Successive Traffic Groundspeed,Successive Traffic Track,Successive Traffic Vertical Rate,Successive Traffic Distance,Successive Traffic Track sin,Successive Traffic Track cos,Successor Aligned,ROT Previous Flight [s],Average ROT Previous 5 Flight [s],Average ROT Previous Flight (Intensity) [s],ROT Previous Flight same A/C Type [s],Average ROT Previous 5 Flights same A/C Type [s],Average ROT Previous Flight same A/C Type (Intensity) [s],Average ROT Previous Flight same A/C Type Total [s],ROT Previous Flight same ICAO Weight Category [s],Average ROT Previous 5 Flights same ICAO Weight Category [s],Average ROT Previous Flight same ICAO Weight Category (Intensity) [s],Average ROT Previous Flight same ICAO Weight Category Total [s],Average ROT Total [s],groundspeed at prediction,geoaltitude at prediction,vertical_rate at prediction,distance at prediction,station,reference_timestamp,temp_2m,temp_5cm,temp_surface,temp_chill,rh_2m,dewpoint_2m,vapour_pressure,pressure_qfe,pressure_qnh,pressure_qff,geopot_850,geopot_700,gust_1s_ms,wind_vec_ms,wind_scalar_ms,wind_dir,foehn_idx,wind_scalar_kmh,gust_3s_ms,gust_1s_kmh,gust_3s_kmh,precip_10min,snow_depth,rad_global,rad_diffuse,rad_lw_in,rad_lw_out,rad_sw_reflect,sunshine_10min,new_wind_speed_kt,new_headwind_kt,new_crosswind_kt,H1,H2,H3
0,SIA346_291674,2024-03-01 06:23:00+00:00,2024-03-01 06:23:57+00:00,SIA,B77W,L2J,Jet,2.0,251290.0,Heavy,North,H1,57.0,2024-03-01 06:21:26+00:00,1.0,0.0,3,5.0,16.0,0.6,30.0,2024-03-01 06:00:00+00:00,1.0,0.5,0.904508,0.206107,0.283058,0.049516,1.0,0.5,0.999013,0.531395,Day,False,True,0.000000,0.00000,1.00,1.00,0,0,0,0,0,7,0,7,0,0,0,0,7,0,7,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1325.000000,0.0,13.752771,0.0,NaN,NaN,16000.0,417.0,317.231857,0.0,19.097551,0.000000,0.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163.0,2950.0,-896.0,4.003528,KLO,2024-03-01 06:20:00+00:00,5.1,4.4,4.6,5.1,97.1,4.7,8.5,959.0,1009.2,1010.3,NaN,NaN,1.5,1.1,1.1,106.0,NaN,4.0,1.5,5.4,5.4,0.4,0.0,2,1.0,336,NaN,NaN,0,2.159827,1.85,-1.12,0,0,0
1,ETH736_1162,2024-03-01 06:32:42+00:00,2024-03-01 06:33:36+00:00,ETH,A359,L2J,Jet,2.0,207000.0,Heavy,South,H1,54.0,2024-03-01 06:31:12+00:00,1.0,0.0,3,5.0,16.0,0.6,30.0,2024-03-01 06:00:00+00:00,1.0,0.5,0.447736,0.002739,0.283058,0.049516,1.0,0.5,0.999013,0.531395,Day,False,True,0.000000,0.00000,1.00,1.00,0,1,0,0,0,9,0,9,0,0,0,0,9,1,10,525.0,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1325.000000,0.0,13.752771,0.0,NaN,NaN,16000.0,417.0,317.231857,0.0,19.097551,0.000000,0.0000

### Save the table

In [108]:
table.to_csv('Final_Table_V2.csv', index=False)